In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from config import gkey, mySQLrp

## Gasoline prices analysis

#### Analyzing gasoline's data from prices and gas stations reported on first two months of 2019

### Function for getting State name based in latitude and longitude

In [2]:
def revGeocoding(lat,long,apiKey):

   url=f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{long}&key={apiKey}"
   response=requests.get(url).json()
    
   #print(response['results'][0]['address_components'][4])
   state=response['results'][1]['address_components'][1]['long_name']

   return state

### Getting prices data

In [3]:
prices_df = pd.read_csv('prices.csv')
prices_df.head()

,place_id,observation,gas_price,type,update_time,Año,Mes,Dia,Hora
0,11703,NaN,20.23,regular,2019-02-15 00:00:00,2019.0,2.0,15.0,0.0
1,11703,NaN,21.49,premium,2019-02-15 00:00:00,2019.0,2.0,15.0,0.0
2,11702,NaN,19.90,regular,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0
3,11702,NaN,21.88,premium,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0
4,11702,NaN,22.25,diesel,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0


In [4]:
prices_df.rename(columns={'place_id':'id_station', 'Año':'year', 'Mes':'month', 'Dia': 'day', 'Hora':'time'},  inplace = True)
prices_df.head()

,id_station,observation,gas_price,type,update_time,year,month,day,time
0,11703,NaN,20.23,regular,2019-02-15 00:00:00,2019.0,2.0,15.0,0.0
1,11703,NaN,21.49,premium,2019-02-15 00:00:00,2019.0,2.0,15.0,0.0
2,11702,NaN,19.90,regular,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0
3,11702,NaN,21.88,premium,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0
4,11702,NaN,22.25,diesel,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0


### Deleting of unnecessary columns

In [5]:
del prices_df['observation']
prices_df.head()

,id_station,gas_price,type,update_time,year,month,day,time
0,11703,20.23,regular,2019-02-15 00:00:00,2019.0,2.0,15.0,0.0
1,11703,21.49,premium,2019-02-15 00:00:00,2019.0,2.0,15.0,0.0
2,11702,19.90,regular,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0
3,11702,21.88,premium,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0
4,11702,22.25,diesel,2019-02-16 01:00:00,2019.0,2.0,16.0,1.0


### Getting gas stations data

In [6]:
stations_df = pd.read_csv('places.csv')
stations_df.head()

,place_id,name,brand,cre_id,category,address_street,x,y
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PEMEX,PL/658/EXP/ES/2015,GAS_STATION,Boulevard Insurgentes No. 1910,-116.92140,32.47641
1,2040,"DIGEPE, S.A. DE C.V. (07356)",PEMEX,PL/902/EXP/ES/2015,GAS_STATION,Carretera Ixmiquilpan - Palmillas Km 58.5 Huic...,-99.74484,20.30370
2,2041,"Diaz Gas, S.A. de C.V.",PEMEX,PL/760/EXP/ES/2015,GAS_STATION,Avenida Adolfo López Mateos No. 1604,-106.45140,31.71947
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PEMEX,PL/825/EXP/ES/2015,GAS_STATION,Ricardo Flores Magón No. 950,-103.30420,20.71413
4,2043,"PETROMAX, S.A. DE C.V.",PEMEX,PL/585/EXP/ES/2015,GAS_STATION,Prolongación Rio Purificación No. 800 Esquina ...,-98.29977,26.03787


In [7]:
stations_df.rename(columns={'place_id':'id_station', 'address_street':'address', 'x':'longitude', 'y': 'latitude'},  inplace = True)
stations_df.head()

,id_station,name,brand,cre_id,category,address,longitude,latitude
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PEMEX,PL/658/EXP/ES/2015,GAS_STATION,Boulevard Insurgentes No. 1910,-116.92140,32.47641
1,2040,"DIGEPE, S.A. DE C.V. (07356)",PEMEX,PL/902/EXP/ES/2015,GAS_STATION,Carretera Ixmiquilpan - Palmillas Km 58.5 Huic...,-99.74484,20.30370
2,2041,"Diaz Gas, S.A. de C.V.",PEMEX,PL/760/EXP/ES/2015,GAS_STATION,Avenida Adolfo López Mateos No. 1604,-106.45140,31.71947
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PEMEX,PL/825/EXP/ES/2015,GAS_STATION,Ricardo Flores Magón No. 950,-103.30420,20.71413
4,2043,"PETROMAX, S.A. DE C.V.",PEMEX,PL/585/EXP/ES/2015,GAS_STATION,Prolongación Rio Purificación No. 800 Esquina ...,-98.29977,26.03787


### Deleting of unnecessary columns

In [8]:
del stations_df['brand']
del stations_df['cre_id']
del stations_df['category']
stations_df.head()

,id_station,name,address,longitude,latitude
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",Boulevard Insurgentes No. 1910,-116.92140,32.47641
1,2040,"DIGEPE, S.A. DE C.V. (07356)",Carretera Ixmiquilpan - Palmillas Km 58.5 Huic...,-99.74484,20.30370
2,2041,"Diaz Gas, S.A. de C.V.",Avenida Adolfo López Mateos No. 1604,-106.45140,31.71947
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",Ricardo Flores Magón No. 950,-103.30420,20.71413
4,2043,"PETROMAX, S.A. DE C.V.",Prolongación Rio Purificación No. 800 Esquina ...,-98.29977,26.03787


### Cleanning gas stations data

In [9]:
stations_df.count()

id_station    12256
name          12256
address       12256
longitude     12256
latitude      12256
dtype: int64

In [10]:
stations_df.dropna(axis=0, how='any', inplace=True)
stations_df.count()

id_station    12256
name          12256
address       12256
longitude     12256
latitude      12256
dtype: int64

In [11]:
len(stations_df.id_station.unique())

12256

### Cleanning prices data

In [12]:
prices_df.count()

id_station     31925
gas_price      31743
type           31743
update_time    31743
year           31743
month          31743
day            31743
time           31743
dtype: int64

In [13]:
prices_df.dropna(axis=0, how='any', inplace=True)
prices_df.count()

id_station     31743
gas_price      31743
type           31743
update_time    31743
year           31743
month          31743
day            31743
time           31743
dtype: int64

In [14]:
len(prices_df.id_station.unique())

11918

### Adding new column to gas stations data and filling it with column name from function revGeocoding

In [15]:
state = 'Mexico'
stations_df.insert(3,'state','')
for i in range(0, 10): #places_df.size):
    #places_df['State'][i] = state
    #print(f"{places_df['y'][i]} - {places_df['x'][i]} - {gkey}")
    state = revGeocoding(stations_df['latitude'][i],stations_df['longitude'][i],gkey)
    #print(state)
    stations_df['state'][i] = state

/Users/tomokot/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
stations_df.head()

,id_station,name,address,state,longitude,latitude
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",Boulevard Insurgentes No. 1910,Avenida Campos,-116.92140,32.47641
1,2040,"DIGEPE, S.A. DE C.V. (07356)",Carretera Ixmiquilpan - Palmillas Km 58.5 Huic...,Hidalgo,-99.74484,20.30370
2,2041,"Diaz Gas, S.A. de C.V.",Avenida Adolfo López Mateos No. 1604,Calle Rubén Posada Pompa,-106.45140,31.71947
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",Ricardo Flores Magón No. 950,Periférico Norte Ricardo Flores Magón Oriente,-103.30420,20.71413
4,2043,"PETROMAX, S.A. DE C.V.",Prolongación Rio Purificación No. 800 Esquina ...,Prolongación Río Purificación,-98.29977,26.03787


### Loading data into local DB

In [19]:
rds_connection_string = 'root:' + mySQLrp + '@127.0.0.1/gasoline_prices'
engine = create_engine(f'mysql://{rds_connection_string}')

In [20]:
stations_df['address'][12169] = 'Avenida Pantitlan 676'

/Users/tomokot/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
stations_df[12168:12170]

,id_station,name,address,state,longitude,latitude
12168,25041,SERVICIO LOMAS AGUASCALIENTES SA DE CV,BLVD. SIGLO XXI No. 3171,,-102.3079,21.84558
12169,25042,"SERVICENTRO PANTITLAN, S.A. DE C.V.",Avenida Pantitlan 676,,-98.9893,19.38131


In [22]:
#stations_df = pd.read_csv("stations_df.csv", encoding="UTF-8") 

In [23]:
#stations_df.columns

In [24]:
#stations_df.drop("Unnamed: 0", axis=1, inplace=True)

In [25]:
stations_df.to_sql(name='station', con=engine, if_exists='append', index=False)

In [26]:
prices_df.to_sql(name='gas_price', con=engine, if_exists='append', index=False)

In [27]:
#stations_df.to_sql(name='station', con=engine, if_exists='append', index=False)

#for i in range(0, stations_df.size):
#    stations_df_tmp = stations_df[i:i+1]
#    stations_df_tmp.to_sql(name='station', con=engine, if_exists='append', index=False)